In [1]:
import os
import numpy as np
import pandas as pd
import cv2
from glob import glob
import tensorflow as tf

from tqdm import tqdm

from keras.layers import *
from keras.applications import MobileNetV2
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers.legacy import Adam

from sklearn.model_selection import train_test_split

In [25]:
if __name__ == "__main__":
   
    dir= os.getcwd()
    path = os.path.join(dir, "dataset")
    
    tr_path = os.path.join(path, "train/*")
    ts_path = os.path.join(path, "test/*")
    labels_path = os.path.join(path, "labels.csv")

    labels_df = pd.read_csv(labels_path)
    breed = labels_df["breed"].unique()
    print("Number of breeds: ", len(breed))

    breed2id = { name: i for i, name in enumerate(breed) }

    ids = glob(tr_path)

    labels = []

    for image_id in ids:
        image_id = image_id.split("/")[-1].split(".")[0]
        print(image_id)

        breed_name = list(labels_df[labels_df.id == image_id]["breed"])[0]
        print(image_id, breed_name)

        breed_index = breed2id[breed_name]
        labels.append(breed_index)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/riaacordero/Documents/dev/dog-breed-identifier-using-tensorflow/dog-breed-identifier-using-tensorflow/dataset/labels.csv'

In [3]:
x_tr, x_ts = train_test_split(ids, test_size=0.2, random_state=42)
y_tr, y_ts = train_test_split(labels, test_size=0.2, random_state=42)

size = 224
num_classes = len(breed)
lr = 1e-4
batch = 8
epochs = 15

In [4]:
def build_model (size, num_classes):
    inputs = Input((size, size, 3))
    backbone = MobileNetV2(input_tensor=inputs, weights='imagenet', include_top=False)
    backbone.trainable = True

    x = backbone.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.2)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs, x)
    return model

In [5]:
model = build_model(size, num_classes)
model.compile(optimizer=Adam(lr), loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']        

In [6]:
def img_read(path, size):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (size, size))
    img = img/255.0
    img = img.astype(np.float32)

    return img

In [7]:
def parse_data(x,y):
    x = x.decode()

    num_class = 120
    size = 224

    img = img_read(x, size)
    label = [0] * num_class
    label[y] = 1
    label = np.array(label)
    label = label.astype(np.int32)

    return img, label

In [8]:
def tf_parse(x, y):
    x, y = tf.numpy_function(parse_data, [x, y], [tf.float32, tf.int32])
    x.set_shape((size, size, 3))
    y.set_shape((num_classes))

    return x, y

In [9]:
def tf_ds(x, y, batch=8):
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.map(tf_parse)
    ds = ds.batch(batch)
    return ds

In [10]:
tr_dataset = tf_ds(x_tr, y_tr, batch=batch)
ts_dataset = tf_ds(x_ts, y_ts, batch=batch)

for x, y in tr_dataset:
    print(x.shape)
    print(y.shape)

(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224, 3)
(8, 120)
(8, 224, 224

KeyboardInterrupt: 

In [ ]:
ids = ids[:1000]
labels = labels[:1000]

callbacks = [
    ModelCheckpoint("model.h5", verbose=1, save_best_only=True),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=1e-6)
]

tr_steps = (len(x_tr) // batch) + 1
ts_steps = (len(x_ts) // batch) + 1


model.fit(tr_dataset, 
          steps_per_epoch=tr_steps,
          validation_steps=ts_steps,
          epochs=epochs,
          validation_data=ts_dataset,
          callbacks=callbacks)

Epoch 1/15


1023/1023 [==============================] - ETA: 0s - loss: 2.8667 - acc: 0.3111
Epoch 1: val_loss improved from inf to 1.66155, saving model to model.h5
1023/1023 [==============================] - 245s 238ms/step - loss: 2.8667 - acc: 0.3111 - val_loss: 1.6615 - val_acc: 0.5222 - lr: 1.0000e-04
Epoch 2/15


/Users/riaacordero/miniconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1023/1023 [==============================] - ETA: 0s - loss: 1.1240 - acc: 0.6776
Epoch 2: val_loss improved from 1.66155 to 1.37351, saving model to model.h5
1023/1023 [==============================] - 249s 243ms/step - loss: 1.1240 - acc: 0.6776 - val_loss: 1.3735 - val_acc: 0.6005 - lr: 1.0000e-04
Epoch 3/15
1023/1023 [==============================] - ETA: 0s - loss: 0.3940 - acc: 0.9011
Epoch 3: val_loss did not improve from 1.37351
1023/1023 [==============================] - 254s 248ms/step - loss: 0.3940 - acc: 0.9011 - val_loss: 1.3839 - val_acc: 0.6298 - lr: 1.0000e-04
Epoch 4/15
1023/1023 [==============================] - ETA: 0s - loss: 0.1529 - acc: 0.9680
Epoch 4: val_loss did not improve from 1.37351
1023/1023 [==============================] - 248s 242ms/step - loss: 0.1529 - acc: 0.9680 - val_loss: 1.4859 - val_acc: 0.6103 - lr: 1.0000e-04
Epoch 5/15
1023/1023 [==============================] - ETA: 0s - loss: 0.1412 - acc: 0.9642
Epoch 5: val_loss did not improve fr

In [32]:
id2breed = {i: name for i, name in enumerate(breed)}

model = tf.keras.models.load_model("model.h5")

for i, path in tqdm(enumerate(x_ts[:50])):
    img = img_read(path,224)
    img = np.expand_dims(img, axis=0)
    pred = model.predict(img)[0]

    label_index = np.argmax(pred)
    breed_name = id2breed[label_index]

    orig_breed = id2breed[y_ts[i]]
    orig_img = cv2.imread(path, cv2.IMREAD_COLOR)

    orig_img = cv2.putText(orig_img, breed_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1)
    orig_img = cv2.putText(orig_img, orig_breed, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    if not os.path.exists(os.path.join(dir, "output")):
        os.makedirs(os.path.join(dir, "output"))
        
    cv2.imwrite(os.path.join(dir, "output", f"{i}.jpg"), orig_img)

0it [00:00, ?it/s]

1/1 [==============================] - 0s 289ms/step


1it [00:00,  3.19it/s]

1/1 [==============================] - 0s 19ms/step


4it [00:00, 10.78it/s]

...

46it [00:02, 26.67it/s]

1/1 [==============================] - 0s 19ms/step


49it [00:02, 26.96it/s]

1/1 [==============================] - 0s 29ms/step


50it [00:02, 22.34it/s]
